In [93]:
A=np.array([[2,0,-1],[4,-1,-2],[3,-1,-1]])

In [88]:
B=A.copy()
n=B.shape[0]
L_inv=np.identity(n)
for j in range(1,n-1+1):
    i=j+1
    for k in range(2,n+1):
        E=np.identity(n)
        E[i-1,j-1]=-B[i-1,j-1]/B[j-1,j-1]
        L_inv=np.matmul(E,L_inv)
        B=np.matmul(E,B)
#         print(i,j)
#         print(B,'\n')
        i+=1
        if i>n:
            break

            
for j in range(n-1+1,1,-1):
    i=j-1
    for k in range(n+1,2,-1):
        E=np.identity(n)
        E[i-1,j-1]=-B[i-1,j-1]/B[j-1,j-1]
        L_inv=np.matmul(E,L_inv)
        B=np.matmul(E,B)
        print(i,j)
        print(B,'\n')
        i-=1
        if i<1:
            break            
            
            
            
            
            
            
            
            
            
            
            
            
            

# U=B.copy()
# print('A=',A)
# print('U=',U)
print('L_inv=',L_inv)

2 3
[[1.  0.  0. ]
 [0.  2.  0. ]
 [0.  0.  1.5]] 

1 3
[[1.  0.  0. ]
 [0.  2.  0. ]
 [0.  0.  1.5]] 

1 2
[[1.  0.  0. ]
 [0.  2.  0. ]
 [0.  0.  1.5]] 

L_inv= [[ 1.          0.          0.        ]
 [-1.33333333  1.33333333 -0.66666667]
 [ 0.5        -0.5         1.        ]]


In [89]:
for i in range(1,n+1):
    L_inv[i-1,:]=L_inv[i-1,:]/B[i-1,i-1]





L_inv

array([[ 1.        ,  0.        ,  0.        ],
       [-0.66666667,  0.66666667, -0.33333333],
       [ 0.33333333, -0.33333333,  0.66666667]])

# Python Assignment 1

### Prof. S. Amini

### Mathematical Methods In Engineering - 1401-01

### Deadline: ?

---

This assignment consists of two problems: In the first one, you're going to use Gauss-Jordan elimination to calculate `LU` factorization of a given matrix, and in the second problem you'll use the same technique to calculate the inverse of a matrix.

Each problem has two cells: A `Code Cell`, and an `Evaluation Cell`. You should write your code in the `Code Cell`s and then run the `Evaluation Cell` to check the correctness of your code. **Please do not edit the codes in the Evaluation Cells.**

Before jumping into the code, read the following section to get familiar with `Gauss transformations`, which helps you do eliminations more efficiently.

## Gauss Transformation

As you have learned in the class, in `Gauss-Jordan elimination`, each time we choose a pivot, we try to eliminate the elements below that pivot. This can be achieved using something called `Gauss Transformation`.

Consider an arbitrary vector $\textbf{v}$ where $v_k \neq 0$

$$ \textbf{v} = 
\begin{bmatrix}
v_1 \\
\vdots \\
v_{k-1} \\
v_k \\
v_{k+1} \\
\vdots \\
v_n
\end{bmatrix}
$$


We need a matrix such when applied to $\textbf{v}$, the elements below $v_k$ become zero. You can check that the `Gauss Matrix` $\textbf{M}_k$, defined as below, does this for us.


$$ \textbf{t}^{(k)} = 
\begin{bmatrix}
0 \\
\vdots \\
0 \\
t_{k+1} \\
\vdots \\
t_n
\end{bmatrix}, \; \; \;
t_i = \frac{v_i}{v_k} \; \; \; \; for \; \; i=k+1:n
$$

$$ \textbf{M}_k = \textbf{I}_n - \textbf{t}^{(k)}\textbf{e}_k^T $$

Where $\textbf{e}_k$ is the k-th column of the $n \times n$ identity matrix $\textbf{I}_n$. Using the matrix-vector multiplication intuition you've learned in the class, you can easily check that the following equality holds:

$$ \textbf{M}_k \textbf{v} = 
\begin{bmatrix}
1 & \ldots & 0 & 0 & \ldots & 0 \\
\vdots && \vdots & \vdots && \vdots \\
0 && 1 & 0 && 1 \\
0 & \ldots & -t_{k+1} & 1 & \ldots & 0 \\
\vdots && \vdots & \vdots && \vdots \\
0 & \ldots & -t_n & 0 & \ldots & 1 \\
\end{bmatrix}
%
\begin{bmatrix}
v_1 \\
\vdots \\
v_{k-1} \\
v_k \\
v_{k+1} \\
\vdots \\
v_n
\end{bmatrix} = 
%
\begin{bmatrix}
v_1 \\
\vdots \\
v_{k-1} \\
v_k \\
0 \\
\vdots \\
0
\end{bmatrix}
$$

The matrix $\textbf{M}_k$ is called a `Gauss matrix`, and the vector $ \textbf{t}^{(k)} $ is called the `Gauss vector`.

---

Before start coding, think about the following questions carefully, as you will use these relations in your code. You don't need to provide their answers in your homework, they're merely for your better understanding.

### Question 1
Investigate what happens to the rows of a matrix if you multiply it by a Gauss matrix?
In other words, if $\textbf{B} = \textbf{M}_k \textbf{A}$, what's the relation between $\textbf{B}[i, :]$ and $\textbf{A}[i, :]$ ?

### Question 2
Show that $$ \textbf{M}_k^{-1} = \textbf{I}_n + \textbf{t}^{(k)}\textbf{e}_k^T $$

---

In [58]:
# import required packages
import numpy as np
import time

## 1. LU Factorization

**Definition**
An $n \times n$ matrix $\textbf{A}$ is said to have an LU decomposition if and only if there exist a lower triangular $n \times n$ matrix $\textbf{L}$ and an upper triangular $n \times n$ matrix $\textbf{U}$ such that:

$$ \textbf{A} = \textbf{LU} $$

Please keep in mind that not all square matrices have LU decomposition, and it may be necessary to permute the rows of a matrix before obtaining its LU factorization. In this problem, we assume that the elimination process does not require any row permutation.

In [73]:
# Code Cell

def LU_factorization(A):
    
    """
        Computes LU factorization for the given matrix
        
        inputs:
            A (numpy ndarray): input matrix
        
        outputs:
            L (numpy ndarray): a lower triangular matrix
            U (numpy ndarray): an upper triangular matrix
    """
    B=A.copy()
    n=B.shape[0]
    # i=0
    # j=0
    L_inv=np.identity(n)
    for j in range(1,n-1+1):
        i=j+1
        for k in range(2,n+1):
            E=np.identity(n)
            E[i-1,j-1]=-B[i-1,j-1]/B[j-1,j-1]
            L_inv=np.matmul(E,L_inv)
            B=np.matmul(E,B)
    #         print(i,j)
    #         print(B,'\n')
            i+=1
            if i>n:
                break

    L=np.linalg.inv(L_inv)
    U=B.copy()
    # print('A=',A)
    # print('U=',U)
    # print('L=',L)
    return L, U

In [74]:
# Evaluation Cell

n = 10
A = np.random.randn(n, n)
s = time.time()
L, U = LU_factorization(A.copy())
elapsed = time.time() - s

assert np.linalg.norm(L - np.tril(L)) < 1e-10, "L is not lower triangular"
assert np.linalg.norm(U - np.triu(U)) < 1e-10, "U is not upper triangular"
assert np.linalg.norm(A - L @ U) < 1e-10, "L and U does not satisfy A = L @ U"

print(f'status: successful, time elapsed: {np.round(elapsed, 5)} seconds')

status: successful, time elapsed: 0.002 seconds


## 2. Inverse of Matrix

**Definition**
Let $\textbf{A}$ be an $n \times n$ matrix. Its inverse, if it exists, is the $n \times n$ matrix $\textbf{A}^{-1}$ such that

$$ \textbf{AA}^{-1} = \textbf{I} $$

Where $\textbf{I}$ is the $n \times n$ identity matrix. If $\textbf{A}^{-1}$ exists, we say that $\textbf{A}$ is invertible. Similar to the previous section, not all square matrices are invertible, but you don't need to worry about this issue in this problem.

In [92]:
def inverse(A):
    B=A.copy()
    n=B.shape[0]
    L_inv=np.identity(n)
    for j in range(1,n-1+1):
        i=j+1
        for k in range(2,n+1):
            E=np.identity(n)
            E[i-1,j-1]=-B[i-1,j-1]/B[j-1,j-1]
            L_inv=np.matmul(E,L_inv)
            B=np.matmul(E,B)
    #         print(i,j)
    #         print(B,'\n')
            i+=1
            if i>n:
                break


    for j in range(n-1+1,1,-1):
        i=j-1
        for k in range(n+1,2,-1):
            E=np.identity(n)
            E[i-1,j-1]=-B[i-1,j-1]/B[j-1,j-1]
            L_inv=np.matmul(E,L_inv)
            B=np.matmul(E,B)
    #         print(i,j)
    #         print(B,'\n')
            i-=1
            if i<1:
                break            

    for i in range(1,n+1):
        L_inv[i-1,:]=L_inv[i-1,:]/B[i-1,i-1]
    


    A_inv=L_inv.copy()    
    return A_inv

In [95]:
# Evaluation Cell

n = 10
A = np.random.rand(n, n) + n * np.eye(n, n)
s = time.time()
A_inv = inverse(A.copy())
elapsed = time.time() - s

assert np.linalg.norm(A @ A_inv - np.eye(n, n)) < 1e-10, "A_inv does not satisfy A @ A_inv = I"

print(f'status: successful, time elapsed: {np.round(elapsed, 5)} seconds')

status: successful, time elapsed: 0.004 seconds
